In [1]:
import pandas as pd
import os
import json
import csv

In [9]:
consistency_list = [1]*41
description_list = [-1]*41
specificities_consistency = {0:0, 2:0, 8:0, 9:0, 11:0, 13:0, 17:0, 19:0, 21:0, 25:0, 27:0, 28:0, 30:0, 37:0, 38:0, 39:0}
specificities_description = {0:"chinese", 2:"chinese", 5:"Key points per utterance", 8:"chinese", 9:"Repeats the conversation", 11:"chinese", 13:"full chinese text", 17:"full chinese text", 19:"full chinese text", 21:"full chinese text", 25:"full chinese text", 27:"full chinese text", 28:"takes back the discourse", 30:"takes back the discourse", 37:"full chinese text", 38:"Sort 2 mots", 39:"full chinese text"}

for key in specificities_consistency.keys():
    consistency_list[key] = specificities_consistency[key]
for key in specificities_description.keys():
    description_list[key] = specificities_description[key]

path_scores_file = "results/" + "2" + "/scores.csv"
dfFINAL = pd.read_csv(path_scores_file)
dfFINAL["consistency"] = consistency_list
dfFINAL["description"] = description_list
dfFINAL.to_csv('scores.csv', encoding='utf-8')
#nb_batch = "1"
#path_scoresINIT_file = "results/" + nb_batch + "/scores.csv"
#path_scoresFINAL_file = "results/" + nb_batch + "/scores4.csv"
#dfFINAL = pd.read_csv(path_scoresFINAL_file)
#dfINIT = pd.read_csv(path_scoresINIT_file)
#dfFINAL["consistency"] = dfINIT["consistency"]
#dfFINAL["description"] = dfINIT["description"]
#dfFINAL.to_csv('scores.csv', encoding='utf-8')

In [2]:
consistency_list = [1]*25
description_list = [-1]*25
specificities_consistency = {3:0, 4:0, 9:0, 10:0, 12:0, 15:0, 16:0, 17:0, 19:0, 20:0, 23:0, 24:0}
specificities_description = {2:"Too long, takes back the text but with hallucinations", 3:"Repeats the text", 4:"Repeats the text", 5:"Très bon résumé", 6:"Statement en une ligne du modèle, pas compris l'instruction et dit n'imp", 7:"Beaucoup trop court, trop incomplet", 8:"Une phrase répétée 2 fois", 9:"Répétitions du texte", 10:"Répétitions du texte", 15:"Répétitions du texte", 12:"Répétitions du texte", 16:"Répétitions du texte", 17:"Répétitions du texte", 19:"Répétitions du texte", 20:"Répétitions du texte", 23:"Répétitions du texte", 24:"Répétitions du texte"}

for key in specificities_consistency.keys():
    consistency_list[key] = specificities_consistency[key]
for key in specificities_description.keys():
    description_list[key] = specificities_description[key]

path_scores_file = "results/" + "3" + "/scores.csv"
dfFINAL = pd.read_csv(path_scores_file)
dfFINAL["consistency"] = consistency_list
dfFINAL["description"] = description_list
dfFINAL.to_csv('scores.csv', encoding='utf-8')

### XGEN
#### 4bit

In [14]:

batches = {'0', '1', '2'}

for nb_batch in batches:
    name_file = "results/" + nb_batch + "/desc.txt"
    file = open(name_file, 'r', encoding='utf-8')
    description = file.readline()
    print("Batch " + nb_batch + ": " + description)
    # Je souhaite obtenir le dataset des samples cohérents -> filtrage
    path_scores_file = "results/" + nb_batch + "/scores.csv"
    # Extract consistency and description
    df = pd.read_csv(path_scores_file)
    array = df.values

    consistent_df = df[df["consistency"]==1]

    consistent_df_en = consistent_df[consistent_df["output_language"]==1]
    consistent_df_en_en = consistent_df_en[consistent_df_en["input_language"]==1]

    consistent_df_en_fr = consistent_df_en[consistent_df_en["input_language"]==0] # Réponses anglaises à une question française

    consistent_df_fr = consistent_df[consistent_df["output_language"]==0]
    consistent_df_fr_fr = consistent_df_fr[consistent_df_fr["input_language"]==0]

    print("Le batch " + nb_batch + " contient " + str(len(consistent_df)) + "/" + str(len(df)) + " échantillons cohérents.")
    print("Parmi les réponses écrites en français, " + str(len(consistent_df_fr_fr)) + "/" + str(len(consistent_df_fr)) + " des questions sont écrites en français")
    print("Parmi les réponses écrites en anglais, " + str(len(consistent_df_en_en)) + "/" + str(len(consistent_df_en)) + " des questions sont écrites en anglais")
    print()

    rouge2_en_fr = consistent_df_en_fr['rouge2'].mean()
    rougel_en_fr = consistent_df_en_fr['rougel'].mean()
    bert_en_fr = consistent_df_en_fr['bertscore'].mean()

    rouge2_fr_fr = consistent_df_fr_fr['rouge2'].mean()
    rougel_fr_fr = consistent_df_fr_fr['rougel'].mean()
    bert_fr_fr = consistent_df_fr_fr['bertscore'].mean()

    rouge2_en_en = consistent_df_en_en['rouge2'].mean()
    rougel_en_en = consistent_df_en_en['rougel'].mean()
    bert_en_en = consistent_df_en_en['bertscore'].mean()

    print("Moyennes questions françaises, réponses françaises (" + str(len(consistent_df_fr_fr)) + " échantillons):")
    print("Rouge 2: " + str(rouge2_fr_fr) + " ---- Rouge L: " + str(rougel_fr_fr) + " ---- BertScore: " + str(bert_fr_fr))
    print()
    print("Moyennes questions anglaises, réponses anglaises (" + str(len(consistent_df_en_en)) + " échantillons):")
    print("Rouge 2: " + str(rouge2_en_en) + " ---- Rouge L: " + str(rougel_en_en) + " ---- BertScore: " + str(bert_en_en))
    print()
    print("Moyennes questions françaises, réponses anglaises (" + str(len(consistent_df_en_fr)) + " échantillons):")
    print("Rouge 2: " + str(rouge2_en_fr) + " ---- Rouge L: " + str(rougel_en_fr) + " ---- BertScore: " + str(bert_en_fr))
    print()

    consistent_df_en_en["compression_rate"] = consistent_df_en_en["nb_words_generated"] / consistent_df_en_en["nb_words_input"]
    consistent_df_en_fr["compression_rate"] = consistent_df_en_fr["nb_words_generated"] / consistent_df_en_fr["nb_words_input"]
    consistent_df_fr_fr["compression_rate"] = consistent_df_fr_fr["nb_words_generated"] / consistent_df_fr_fr["nb_words_input"]
    compression_rate_en_en = consistent_df_en_en["compression_rate"].mean()
    compression_rate_en_fr = consistent_df_en_fr["compression_rate"].mean()
    compression_rate_fr_fr = consistent_df_fr_fr["compression_rate"].mean()

    print("Taux de compression questions anglaises réponses anglaises: " + str(compression_rate_en_en))
    print("Taux de compression questions françaises réponses anglaises: " + str(compression_rate_en_fr))
    print("Taux de compression questions françaises réponses françaises: " + str(compression_rate_fr_fr))

    print()
    print()
    # Grosse tendance à écrire en anglais. Et c'est normal, le header de finetuning est en anglais... Les tâches de finetuning ne sont pas multilingues !
    # Du coup, problème lors du calcul des scores: calculs sur du gold fr et du generated en. De plus, problème plus général sur les métriques, un bertscore français est différent d'un bertscore anglais, pas comparable et la moyenne du rouge score entre deux textes randoms en langue anglais et française est probablement différente... Limites de ces mesures. Justifie l'analyse qualitative.


Batch 1: This inference batch is aimed at testing XGen 8bit on summarization. We shall compare his performance on French and on English
Le batch 1 contient 39/40 échantillons cohérents.
Parmi les réponses écrites en français, 7/7 des questions sont écrites en français
Parmi les réponses écrites en anglais, 20/32 des questions sont écrites en anglais

Moyennes questions françaises, réponses françaises (7 échantillons):
Rouge 2: 0.06477100407250354 ---- Rouge L: 0.15354240474238873 ---- BertScore: -0.001854958121839447

Moyennes questions anglaises, réponses anglaises (20 échantillons):
Rouge 2: 0.0938697219794203 ---- Rouge L: 0.1929056223867459 ---- BertScore: 0.24701896104961635

Moyennes questions françaises, réponses anglaises (12 échantillons):
Rouge 2: 0.04485129846821542 ---- Rouge L: 0.14029715907759835 ---- BertScore: 0.13658359218000743

Taux de compression questions anglaises réponses anglaises: 0.3851305874064335
Taux de compression questions françaises réponses anglaises: 0

C:\Users\rayci\AppData\Local\Temp\ipykernel_88320\2079108961.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consistent_df_en_en["compression_rate"] = consistent_df_en_en["nb_words_generated"] / consistent_df_en_en["nb_words_input"]
C:\Users\rayci\AppData\Local\Temp\ipykernel_88320\2079108961.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consistent_df_en_fr["compression_rate"] = consistent_df_en_fr["nb_words_generated"] / consistent_df_en_fr["nb_words_input"]
C:\Users\rayci\AppData\Local\Temp\ip